In [1]:
import numpy as np 
from game import Game, Player, Move, Board
from copy import deepcopy

In [2]:
g = Game()
g.get_board()

array([[-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1]], dtype=int8)

In [3]:
def score(player, state : Board) :
    board = state.board
    score = 5
    for row in board :
        new_score = np.count_nonzero(row == player)
        score = min(score, 5 - new_score)
    
    for col in range(board.shape[1]) :
        new_score = np.count_nonzero(board[:,col] == player)
        score = min(score, 5 - new_score)
    
    diag1 = board.diagonal()
    new_score = np.count_nonzero(diag1 == player)
    score = min(score, 5 - new_score)

    diag2 = np.fliplr(board).diagonal()
    new_score = np.count_nonzero(diag2 == player)
    score = min(score, 5 - new_score)

    return score

In [4]:
def get_periphery_board(board) :
    shape_y, shape_x = board.shape
    periphery_cubes = set()
    for i in range(shape_x) :
        periphery_cubes.add((0,i))
        periphery_cubes.add((shape_y-1,i))
    for j in range(shape_y) :
        periphery_cubes.add((j,0))
        periphery_cubes.add((j, shape_x-1))
    
    return periphery_cubes

In [5]:
def get_successors(player, board : Board) :
    '''Check which cube can be played'''
    playable_cubes = []
    for pos in get_periphery_board(board) :
        if board[pos] == -1 or board[pos] == player :
            playable_cubes.append(pos)
    
    '''For each playable cube, compute the possible successors, depending on the movement of slide, and returns the move and the associated board'''
    successors = []
    for pos in playable_cubes :
        for slide_direction in board.acceptable_slides(pos) :
            new_board = deepcopy(board)
            new_board[pos] = player
            new_board.slide(pos, slide_direction)
            successors.append([(pos, slide_direction), new_board.board])
        
    return successors
    

In [21]:
class Tree :
    def __init__(self, board = None, children : list = None) -> None:
        self.board = board
        self.children = children if children is not None else dict()
        # Moves are the plays corresponding one to one to the children boards
        self.score = -1        
    
    def get_leaves(self) :
        if self.children == dict() :
            return [self]
        leaves = []
        for move, node in self.children.items() :
            leaves.extend(node.get_leaves())
        
        return leaves
    
'''Computes all the next possible moves and boards until a given depth is reached, for further application of MinMax'''
def get_states_tree(player : int, board : Board, max_depth : int, initial_player : int) :
    tree = Tree(board)
    if board.check_winner() != -1 or max_depth == 0 :
        return tree

    for move, succ in get_successors(player, board) :
        tree.children[move] = get_states_tree(1 - player, Board(succ), max_depth - 1, initial_player)

    return tree

'''Computes the score of the leaves of the tree (furthest anticipated moves) for further application of MinMax'''
def valuate_tree(player : int, states_tree : Tree, score_function = score) :
    valuated_tree = deepcopy(states_tree)
    for leaf in valuated_tree.get_leaves() :
        leaf.score = score(player, leaf.board)
    
    return valuated_tree

In [22]:
'''MinMax Algorithm where the root node is always the player who tries to maximize the score --> Returns the Move as well as the score, as we want to know what to play'''
def min_max(valuated_tree : Tree, compute_max=False) :
    if valuated_tree.children == dict() :
        return None, valuated_tree.score
    
    options = []
    for move, child in valuated_tree.children.items() :
        options.append([move, min_max(child, compute_max=bool(1-compute_max))[1]])
    
    if compute_max :
        return max(options, key = lambda t : t[1])
    else : 
        return min(options, key = lambda t : t[1])
    

In [24]:
class MinMaxPlayer(Player) :
    def __init__(self, score_function = score) -> None:
        super().__init__()
        self.score_function = score_function
    
    def make_move(self, game: Game) -> tuple[tuple[int, int], Move]:
        board = Board(game.get_board())
        player = game.get_current_player()
        tree = get_states_tree(player, board, 2, player)
        value_tree = valuate_tree(player, tree, self.score_function)
        move, score = min_max(value_tree)
        return move
        

In [28]:
from main import RandomPlayer

g = Game()
g.play(RandomPlayer(), RandomPlayer())

1

## TODO :
- ~~In get_successors, return the Moves associated to the next boards~~
- ~~Store the Move associated to the Board in the Tree (In the form of a Dict[move : child])~~
- ~~Change MinMax to work on dict[child : score of MinMax] to be able to return a direct child of the root and not a leaf of the tree~~
- Optimize trees generation to have quicker games (Try caching and/or combine tree generation and valuation to have Alpha-Beta pruning during generation)
- Improve score function to be more performant
- See if it is possible to adapt the max_depth of the Tree based on the device capabilities
- Add the possibility to play against AI

In [8]:
g = Game()
t = get_states_tree(0, Board(g.get_board()), 2, 0)
v = valuate_tree(0, t)
min_max(v, True)

[((0, 1), <Move.BOTTOM: 1>), 4]

In [20]:
board = np.array([[ 0, 0, 0, 1,-1],
                  [ 1,-1,-1,-1, 0],
                  [ 0, 1, 1,-1,-1],
                  [ 1, 0,-1,-1,-1],
                  [ 0, 1, 1, 0,-1]])
t = get_states_tree(0, Board(board), 3, 0)
v = valuate_tree(0, t)
min_max(v, compute_max=False)

[((0, 1), <Move.BOTTOM: 1>), 2]

In [ ]:
s = get_states_tree(0, Board(board), 3, 0)
v = valuate_tree(0,s)
for succ in v.children.items() :
    print(succ[1].score)

In [11]:
max([1,2,3])

3